In [1]:
from sympy import false, true
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pickle

In [4]:
# Data Import 

# Load X_train
with open("data/X_train.pkl", "rb") as f:
    X_train = pickle.load(f)

# Load y_train
with open("data/y_train.pkl", "rb") as f:
    y_train = pickle.load(f)

# Load X_val
with open("data/X_val.pkl", "rb") as f:
    X_val = pickle.load(f)

# Load y_val
with open("data/y_val.pkl", "rb") as f:
    y_val = pickle.load(f)

# Load X_test
with open("data/X_test.pkl", "rb") as f:
    X_test = pickle.load(f)

# Load y_test
with open("data/y_test.pkl", "rb") as f:
    y_test = pickle.load(f)

# Load df_final_viz
with open("data/df_final_viz.pkl", "rb") as f:
    df_final_viz = pickle.load(f)

In [32]:
# In Torch-Tensoren umwandeln
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Datasets und DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last = True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last = True)

In [31]:
X_train.shape[0]

27024

In [33]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dense_units):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(hidden_size, dense_units)
        self.fc2 = nn.Linear(dense_units, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # Nur der letzte Zeitschritt
        out = self.dropout(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
y_batch.shape

torch.Size([32, 1])

In [36]:
rnn_model = SimpleRNN(input_size = X_train.shape[2], hidden_size = 48, dense_units = 32)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(rnn_model.parameters(), lr = 0.001)

num_epochs = 50
for epoch in range(num_epochs):
    rnn_model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        y_pred = rnn_model(X_batch)
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss/= len(train_loader)

    rnn_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = rnn_model(X_batch)
            loss = criterion(y_pred, y_batch)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')


Epoch [1/50], Train Loss: 0.0049, Val Loss: 0.0011
Epoch [2/50], Train Loss: 0.0016, Val Loss: 0.0008
Epoch [3/50], Train Loss: 0.0012, Val Loss: 0.0007
Epoch [4/50], Train Loss: 0.0010, Val Loss: 0.0007
Epoch [5/50], Train Loss: 0.0009, Val Loss: 0.0007
Epoch [6/50], Train Loss: 0.0009, Val Loss: 0.0006
Epoch [7/50], Train Loss: 0.0008, Val Loss: 0.0007
Epoch [8/50], Train Loss: 0.0008, Val Loss: 0.0009
Epoch [9/50], Train Loss: 0.0008, Val Loss: 0.0007
Epoch [10/50], Train Loss: 0.0008, Val Loss: 0.0006
Epoch [11/50], Train Loss: 0.0008, Val Loss: 0.0009
Epoch [12/50], Train Loss: 0.0008, Val Loss: 0.0006
Epoch [13/50], Train Loss: 0.0007, Val Loss: 0.0007
Epoch [14/50], Train Loss: 0.0007, Val Loss: 0.0008
Epoch [15/50], Train Loss: 0.0007, Val Loss: 0.0008
Epoch [16/50], Train Loss: 0.0007, Val Loss: 0.0008
Epoch [17/50], Train Loss: 0.0007, Val Loss: 0.0006
Epoch [18/50], Train Loss: 0.0007, Val Loss: 0.0006
Epoch [19/50], Train Loss: 0.0007, Val Loss: 0.0006
Epoch [20/50], Train 

In [10]:
X_batch.shape

torch.Size([16, 24, 17])